In [28]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor

import sys 
import os
sys.path.append(os.path.abspath('../models'))
sys.path.append(os.path.abspath('../dev'))

preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

trainstreamer = train.stream()
validstreamer = valid.stream()

2025-05-27 18:03:39.375 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 2791.73it/s]


In [29]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy
import torch

accuracy = Accuracy()
loss_fn = torch.nn.CrossEntropyLoss()

In [30]:
from pathlib import Path


settings = TrainerSettings(
    epochs=10, 
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 5, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 10
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 5, 'delta': 0.0}

In [35]:
import timeit
import mlflow
from datetime import datetime

from mltrainer import Trainer
from torch import optim

from RNN import GRUmodel, ModelConfig

mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

with mlflow.start_run():
    mlflow.set_tag("model", "GRU")
    mlflow.set_tag("dev", "melissa")
    config = ModelConfig(
        input_size=3, # vast
        hidden_size=64,
        num_layers=2,
        output_size=20, # vast
        dropout=0.4,
    )

    model = GRUmodel(
        config=config,
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    )
    timeit 
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

2025-05-27 18:06:55.875 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250527-180655
2025-05-27 18:06:55.877 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 81/81 [00:03<00:00, 26.97it/s]
2025-05-27 18:06:59.013 | INFO     | mltrainer.trainer:report:205 - Epoch 0 train 2.6559 test 2.2622 metric ['0.1969']
100%|██████████| 81/81 [00:02<00:00, 34.12it/s]
2025-05-27 18:07:01.520 | INFO     | mltrainer.trainer:report:205 - Epoch 2 train 2.1841 test 2.1000 metric ['0.2219']
100%|██████████| 81/81 [00:02<00:00, 31.42it/s]
2025-05-27 18:07:04.225 | INFO     | mltrainer.trainer:report:205 - Epoch 4 train 2.0261 test 1.8505 metric ['0.3797']
100%|██████████| 81/81 [00:02<00:00, 33.34it/s]
2025-05-27 18:07:06.789 | INFO     | mltrainer.trainer:report:205 - Epoch 6 train 1.6785 test 1.3366 metric ['0.5766']
100%|██████████| 81/81 [00:02<00:00, 30.55it/s]
2025-05-27 18:07:09.580 | INFO     | mltrainer.trainer:report:205 - Epoch 8 train 1.1510 test 0.9738 metric ['0.6547']
 50%|█████     | 5/10 [00:13<00:13,  2.74s/it]


KeyboardInterrupt: 